In [1]:
import numpy as np
import pandas as pd
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType

df = pd.read_csv("C:\\Users\\Leo\\code\\ch04_LoRA\\data\\review_data.csv", encoding="cp949")

train_df, test_df = train_test_split(df, test_size=0.2, random_state=0, stratify=df['labels'])
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

model_name = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# LoRA 설정
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    target_modules=["query", "value"],
)
# LORA 모델 적용
model = get_peft_model(model, peft_config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
#LoRA 로 training
def preprocess(data):
    res =  tokenizer(data['text'], padding='max_length', truncation=True, max_length=64)
    return res

train_dataset = train_dataset.map(preprocess, batched=True)
test_dataset = test_dataset.map(preprocess, batched=True)

training_args = TrainingArguments(
    output_dir="./saved_models/peft_lora_sentiment",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,

    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,

    logging_dir="./logs",
    logging_steps=10,
    label_names=["labels"],
    use_cpu = True,
)

def compute_metrics(predict):
    preds = np.argmax(predict.predictions, axis=1)
    acc = (preds == predict.label_ids).mean()
    return {"accuracy": acc}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    )

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [3]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.785756,0.250000
2,No log,0.778081,0.250000
3,0.767100,0.775263,0.250000


TrainOutput(global_step=12, training_loss=0.7543654044469198, metrics={'train_runtime': 11.113, 'train_samples_per_second': 4.319, 'train_steps_per_second': 1.08, 'total_flos': 1584130498560.0, 'train_loss': 0.7543654044469198, 'epoch': 3.0})

In [4]:
test_texts = ["이 제품 너무 좋아요!", "별로예요. 추천 안함."]
inputs = tokenizer(test_texts, padding=True, truncation=True, max_length=64, return_tensors="pt")
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1)
    print("결과 :", predictions.tolist())  # 1은 긍정, 0은 부정

결과 : [1, 1]


In [5]:
#Lora adapter 저장
model.save_pretrained("./lora_adapter")


In [6]:
# LoRA 모델 불러오기
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel, PeftConfig
peft_config = PeftConfig.from_pretrained("./lora_adapter")

model_name = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
model = PeftModel.from_pretrained(model, "./lora_adapter")
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
test_texts = ["이 제품 너무 좋아요!", "별로예요. 추천 안함."]
inputs = tokenizer(test_texts, padding=True, truncation=True, max_length=64, return_tensors="pt")
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1)
    print("결과 :", predictions.tolist())  # 1은 긍정, 0은 부정

결과 : [1, 1]
